In [6]:
# Model paths
MODEL_TYPE = "gpt2" 
OUTPUT_DIR = f"../../weights/es-gpt2/"
#TRAIN_PATH = f"../../data/spanish-corpora/preprocessed_DOGC_lower.txt"
#TEST_PATH = f"../../data/spanish-corpora/preprocessed_DGT_lower.txt"
#VAL_PATH = f"../../data/spanish-corpora/preprocessed_DOGC_lower.txt"
TRAIN_PATH = f"../../data/minicorpus/train_1.txt"
TEST_PATH = f"../../data/minicorpus/test_1.txt"
VAL_PATH = f"../../data/minicorpus/val_1.txt"

#TRAIN_PATH = f"/home/camilojd/Universidad/ALS/data/minicorpus/train_1.txt"
#TEST_PATH = f"/home/camilojd/Universidad/ALS/data/minicorpus/test_1.txt"
#VAL_PATH = f"./home/camilojd/Universidad/ALS/data/minicorpus/val_1.txt"


# Model paths
#MODEL_TYPE = "gpt2" 
#OUTPUT_DIR = f"../../weights/{MODEL_TYPE}/papers_milan/"
#TRAIN_PATH = f"../../data/papers_milan/train_papers.txt"
#TEST_PATH = f"../../data/papers_milan/test_papers.txt"
#VAL_PATH = f"../../data/papers_milan/val_papers.txt"

In [2]:
!pwd

/home/camilojd/Universidad/ALS/src/finetuning


# Finetune Tokenizer

In [ ]:
from pathlib import Path
from tokenizers import ByteLevelBPETokenizer

#paths = [str(x) for x in Path(TRAIN_PATH).glob("**/*.txt")]
paths = [TRAIN_PATH]
paths

In [ ]:
# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer()

# Customize training
tokenizer.train(files=paths, vocab_size=52_000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

In [ ]:
!mkdir OUTPUT_DIR
tokenizer.save_model(OUTPUT_DIR)

# Finetune Model

In [3]:
# Check that PyTorch sees it
import torch
torch.cuda.is_available()

True

In [4]:
from transformers import GPT2Config

config = GPT2Config()

In [112]:
from transformers import GPT2TokenizerFast

tokenizer = GPT2TokenizerFast.from_pretrained(OUTPUT_DIR)#, max_len=512)

special_tokens_dict = {'bos_token': '<BOS>', 'eos_token': '<EOS>', 'pad_token': '<PAD>'}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)


In [113]:
from transformers import GPT2Model

model = GPT2Model(config=config)
model.num_parameters()

model.resize_token_embeddings(len(tokenizer))

Embedding(52004, 768)

In [96]:
import logging
import os
import pickle
import time

import torch
from filelock import FileLock
from torch.utils.data.dataset import Dataset

from transformers.tokenization_utils import PreTrainedTokenizer

logger = logging.getLogger(__name__)

from nlp import load_dataset
from nlp.builder import FORCE_REDOWNLOAD

class LineByLineNLPTextDataset(Dataset):
    """
    This will be superseded by a framework-agnostic approach
    soon.
    """

    def __init__(self, tokenizer: PreTrainedTokenizer, file_path: str, block_size: int, overwrite_cache=False):
        assert os.path.isfile(file_path)
        # Here, we do not cache the features, operating under the assumption
        # that we will soon use fast multithreaded tokenizers from the
        # `tokenizers` repo everywhere =)
        logger.info("Creating features from dataset file at %s", file_path)
        
        if overwrite_cache:
            ### FIX
            dataset = load_dataset(path='text', 
                       name="dataset",
                       data_files={
                           'train': [file_path]
                       },
                       download_mode=FORCE_REDOWNLOAD,
                       version="0.0.0"
                  )    
        else:
            dataset = load_dataset(path='text', 
                       name="dataset",
                       data_files={
                           'train': [file_path]
                       }
                  )

        dataset = dataset["train"].map(lambda line: tokenizer(
                                            line["text"], 
                                            add_special_tokens=True, 
                                            truncation=True, 
                                            max_length=block_size),
                                  batched=True)
        #dataset.set_format(type='numpy', columns=['input_ids', 'token_type_ids', 'attention_mask'])
        dataset.set_format(type='numpy', columns=['input_ids'])
        
        self.examples = dataset

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i) -> torch.Tensor:
        return torch.tensor(self.examples[i]["input_ids"], dtype=torch.long)


In [66]:
class LineByLineTextDataset(Dataset):
    """
    This will be superseded by a framework-agnostic approach
    soon.
    """

    def __init__(self, tokenizer: PreTrainedTokenizer, file_path: str, block_size: int):
        assert os.path.isfile(file_path)
        # Here, we do not cache the features, operating under the assumption
        # that we will soon use fast multithreaded tokenizers from the
        # `tokenizers` repo everywhere =)
        logger.info("Creating features from dataset file at %s", file_path)

        with open(file_path, encoding="utf-8") as f:
            lines = []
            for line in f.read().splitlines():
                if (len(line) > 0 and not line.isspace()):
                    print(line)
                    lines.append(line)
                    break
            #lines = [line for line in f.read().splitlines() if (len(line) > 0 and not line.isspace())]

        batch_encoding = tokenizer.batch_encode_plus(lines, add_special_tokens=True, max_length=block_size)
        self.examples = batch_encoding["input_ids"]

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i) -> torch.Tensor:
        return torch.tensor(self.examples[i], dtype=torch.long)

In [102]:
#from transformers import LineByLineTextDataset, TextDataset
from nlp.builder import FORCE_REDOWNLOAD

dataset = LineByLineNLPTextDataset(
    tokenizer=tokenizer,
    file_path=TRAIN_PATH,
    block_size=64,
    overwrite_cache=False
)

dataset1 = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path=TRAIN_PATH,
    block_size=64
)

Checking /home/camilojd/.cache/huggingface/datasets/3e34209a2741375a1db1ff03bf1abba1a9bd0e6016912d3ead0114b9d1ca2685.557eb955d95e9c9f3882df6ec794d4f05c1bff683ea7c7e207ab764c7fc9eac3.py for additional imports.
Found main folder for dataset https://s3.amazonaws.com/datasets.huggingface.co/nlp/datasets/text/text.py at /home/camilojd/Universidad/ALS/nlp/src/nlp/datasets/text
Found specific version folder for dataset https://s3.amazonaws.com/datasets.huggingface.co/nlp/datasets/text/text.py at /home/camilojd/Universidad/ALS/nlp/src/nlp/datasets/text/3a79870d85f1982d6a2af884fde86a71c771747b4b161fd302d28ad22adf985b
Found script file from https://s3.amazonaws.com/datasets.huggingface.co/nlp/datasets/text/text.py to /home/camilojd/Universidad/ALS/nlp/src/nlp/datasets/text/3a79870d85f1982d6a2af884fde86a71c771747b4b161fd302d28ad22adf985b/text.py
Couldn't find dataset infos file at https://s3.amazonaws.com/datasets.huggingface.co/nlp/datasets/text/dataset_infos.json
Found metadata file for dataset

aprobada en la 81a. sesión plenaria, celebrada el 4 de diciembre de 2000, por recomendación de la comisión (a/55/602/add.2, párr. 94),el proyecto de resolución recomendado en el informe fue patrocinado en la comisión por los países siguientes: bolivia, cuba, el salvador, ghana y honduras. en votación registrada de 106 votos contra uno y 67 abstenciones, como sigue: 


In [104]:
dataset[0]

tensor([18904, 46048,   286,   283, 11485,    69,    18,  5029,  6214,    16,
         3671,   289,   613,   262,  1121,   262,  1376,    16,   342,  2685,
          262,   283,   839,   358,    69,    19,  5873,    19, 45082,    19,
         7202,    18,    22,    16,  6329,    18,  8411,  1637,   473,  1566,
          262,  2158,  7293,   286,   289,  1290,  1009,  8092,   286,   283,
          839,   342,   300,   762,  1634,    30,  7400,    16,  5349,    16,
          289,  7270,    16,  7676])

In [92]:
dataset1[0]

tensor([18904, 46048,   286,   283, 11485,    69,    18,  5029,  6214,    16,
         3671,   289,   613,   262,  1121,   262,  1376,    16,   342,  2685,
          262,   283,   839,   358,    69,    19,  5873,    19, 45082,    19,
         7202,    18,    22,    16,  6329,    18,  8411,  1637,   473,  1566,
          262,  2158,  7293,   286,   289,  1290,  1009,  8092,   286,   283,
          839,   342,   300,   762,  1634,    30,  7400,    16,  5349,    16,
          289,  7270,    16,  7676])

In [111]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    overwrite_output_dir=True,
    per_device_train_batch_size=1,
    save_steps=10_000,
    save_total_limit=2,
    warmup_steps=10_000,
    max_steps=900_000,
    learning_rate=0.0001,
    fp16=True
)

from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False, mlm_probability=0.15
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    prediction_loss_only=True,
)

In [110]:
%%time
trainer.train()

TypeError: forward() got an unexpected keyword argument 'labels'

In [ ]:
trainer.save_model(OUTPUT_DIR)